In [1]:
import os

In [2]:
%pwd


'd:\\Inventory-Management-and-Predictive-Analytics-with-IOT\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'd:\\Inventory-Management-and-Predictive-Analytics-with-IOT'

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder


# Categorigal Pipeline
categorical_cols_for_OrdinalEncoding = ['category']

Numerical_col_1  = ['temperature','unit_price', 'timestamp_day_of_month', 'timestamp_day_of_week','timestamp_hour']

Numerical_col_2  = ['quantity']

categorical_col_for_OnehotEncoding = ['category']


Numerical_pipeline_one_for_missingvalues = Pipeline(
                steps=[
                ('scaler',StandardScaler())
                 ])

Numerical_pipeline_two_for_missingvalues = Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='constant', fill_value=0)),
                ('scaler',StandardScaler())
                 ])
            
categorical_pipeline_for_OnehotEncoding = Pipeline(
                            steps=[
                            ('one_hot_encoder', OneHotEncoder(sparse_output=False,handle_unknown = 'ignore')),
                            ('scaler',StandardScaler())
                            ]
                            )

preprocessor=ColumnTransformer(transformers= 
        [('Numerical_pipeline_1_for_missingvalues', Numerical_pipeline_one_for_missingvalues,Numerical_col_1),
         ('Numerical_pipeline_2_for_missingvalues', Numerical_pipeline_two_for_missingvalues,Numerical_col_2),
         ('categorical_pipeline_for_OnehotEncoding', categorical_pipeline_for_OnehotEncoding,categorical_col_for_OnehotEncoding )
         ],
                                                          remainder='passthrough',sparse_threshold=0)     





In [6]:
import pandas as pd

merged_df = pd.read_csv('artifacts/data_validation/validated_data_after_dropping_columns/merged_output.csv')

In [7]:
merged_df.head()

,estimated_stock_pct,quantity,temperature,category,unit_price,timestamp_day_of_month,timestamp_day_of_week,timestamp_hour
0,0.89,3.0,-0.02885,kitchen,11.19,1,1,9
1,0.14,3.0,-0.02885,vegetables,1.49,1,1,9
2,0.67,0.0,-0.02885,baby products,14.19,1,1,9
3,0.82,0.0,-0.02885,beverages,20.19,1,1,9
4,0.05,0.0,-0.02885,pets,8.19,1,1,9


In [8]:
preprocessor.fit_transform(merged_df)


array([[ 0.28450808,  0.41770507, -1.50677743, ..., -0.21799646,
        -0.212604  ,  0.89      ],
       [ 0.28450808, -1.38192843, -1.50677743, ..., -0.21799646,
         4.70358027,  0.14      ],
       [ 0.28450808,  0.97429275, -1.50677743, ..., -0.21799646,
        -0.212604  ,  0.67      ],
       ...,
       [ 0.07481138, -0.36151768,  1.49579523, ..., -0.21799646,
        -0.212604  ,  0.78      ],
       [ 0.07481138,  1.12271613,  1.49579523, ..., -0.21799646,
        -0.212604  ,  0.92      ],
       [ 0.07481138, -0.69547029,  1.49579523, ..., -0.21799646,
        -0.212604  ,  0.01      ]])

In [9]:
df_final = pd.DataFrame(preprocessor.fit_transform(merged_df))


In [10]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.284508,0.417705,-1.506777,-0.995194,-1.578795,1.16999,-0.217067,-0.22936,-0.224191,-0.231579,...,-0.220077,-0.204418,-0.221226,-0.202945,-0.22487,-0.217067,-0.229137,-0.217996,-0.212604,0.89
1,0.284508,-1.381928,-1.506777,-0.995194,-1.578795,1.16999,-0.217067,-0.22936,-0.224191,-0.231579,...,-0.220077,-0.204418,-0.221226,-0.202945,-0.22487,-0.217067,-0.229137,-0.217996,4.703580,0.14
2,0.284508,0.974293,-1.506777,-0.995194,-1.578795,-0.50824,4.606882,-0.22936,-0.224191,-0.231579,...,-0.220077,-0.204418,-0.221226,-0.202945,-0.22487,-0.217067,-0.229137,-0.217996,-0.212604,0.67
3,0.284508,2.087468,-1.506777,-0.995194,-1.578795,-0.50824,-0.217067,-0.22936,-0.224191,4.318187,...,-0.220077,-0.204418,-0.221226,-0.202945,-0.22487,-0.217067,-0.229137,-0.217996,-0.212604,0.82
4,0.284508,-0.138883,-1.506777,-0.995194,-1.578795,-0.50824,-0.217067,-0.22936,-0.224191,-0.231579,...,-0.220077,-0.204418,-0.221226,4.927446,-0.22487,-0.217067,-0.229137,-0.217996,-0.212604,0.05


In [11]:
df_final.shape

(10845, 29)

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_final)

In [13]:
train.shape

(8133, 29)

In [14]:
test.shape

(2712, 29)

In [15]:
import os
from src.mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [16]:

from src.mlProject.constants import *
from src.mlProject.utils.common import *

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_file_path: Path
    test_file_path: Path
    transformed_object_file_path: Path
    transformed_train_file_path: Path
    transformed_test_file_path: Path

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            train_file_path = config.train_file_path,
            test_file_path = config.test_file_path,
            transformed_object_file_path = config.transformed_object_file_path,
            transformed_train_file_path = config.transformed_train_file_path,
            transformed_test_file_path = config.transformed_test_file_path
        )

        return data_transformation_config

In [19]:
from src.mlProject.entity.config_entity import DataTransformationConfig
import os
from src.mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config



    def get_data_transformer_object(self):
        try:

            Numerical_col_1  = ['temperature','unit_price', 'timestamp_day_of_month', 'timestamp_day_of_week','timestamp_hour']

            Numerical_col_2  = ['quantity']

            categorical_col_for_OnehotEncoding = ['category']


            Numerical_pipeline_one_for_missingvalues = Pipeline(
                            steps=[
                            ('scaler',StandardScaler())
                            ])

            Numerical_pipeline_two_for_missingvalues = Pipeline(
                            steps=[
                            ('imputer',SimpleImputer(strategy='constant', fill_value=0)),
                            ('scaler',StandardScaler())
                            ])
                        
            categorical_pipeline_for_OnehotEncoding = Pipeline(
                                        steps=[
                                        ('one_hot_encoder', OneHotEncoder(sparse_output=False,handle_unknown = 'ignore')),
                                        ('scaler',StandardScaler())
                                        ]
                                        )

            preprocessor=ColumnTransformer(transformers= 
                    [('Numerical_pipeline_1_for_missingvalues', Numerical_pipeline_one_for_missingvalues,Numerical_col_1),
                    ('Numerical_pipeline_2_for_missingvalues', Numerical_pipeline_two_for_missingvalues,Numerical_col_2),
                    ('categorical_pipeline_for_OnehotEncoding', categorical_pipeline_for_OnehotEncoding,categorical_col_for_OnehotEncoding )
                    ],
                                                                    remainder='passthrough',sparse_threshold=0)     
            
            return preprocessor
            

        except Exception as e:
            raise e
    
    def initiate_data_transformation(self,):
        try:
            
            train_df = pd.read_csv(self.config.train_file_path)
            test_df = pd.read_csv(self.config.test_file_path)
            preprocessor = self.get_data_transformer_object()


            #training dataframe
            input_feature_train_df = train_df.drop(columns='estimated_stock_pct', axis=1)
            target_feature_train_df = train_df['estimated_stock_pct']

            #testing dataframe
            input_feature_test_df = test_df.drop(columns='estimated_stock_pct', axis=1)
            target_feature_test_df = test_df['estimated_stock_pct']

            preprocessor_object = preprocessor.fit(input_feature_train_df)
            input_feature_train_final = preprocessor_object.transform(input_feature_train_df)
            input_feature_test_final =preprocessor_object.transform(input_feature_test_df)


            train_arr = np.c_[input_feature_train_final, np.array(target_feature_train_df)]
            test_arr = np.c_[input_feature_test_final, np.array(target_feature_test_df)]


            #save numpy array data

            save_numpy_array(train_arr, self.config.transformed_train_file_path, 'train_array.npy')
            save_numpy_array(test_arr, self.config.transformed_train_file_path, 'test_array.npy')
            save_preprocessor(preprocessor_object, self.config.transformed_object_file_path)

            
            return (
                train_arr,
                test_arr,
                self.config.transformed_object_file_path, preprocessor_object
            )
        
        except Exception as e:
            raise e
            



In [20]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transformer_object()
    data_transformation.initiate_data_transformation()

except Exception as e:
    raise e

[2023-11-21 13:22:48,767: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-21 13:22:48,775: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 13:22:48,779: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-21 13:22:48,782: INFO: common: created directory at: artifacts]
[2023-11-21 13:22:48,786: INFO: common: created directory at: artifacts/data_transformation]


: 